Plots Q9

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import gamma
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

# Sample DataFrame
# sprog = pd.DataFrame({'wind_speed': [...], 'wind_direction': [...]})

# Define the directional bins
bins = np.array([0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360])
labels = np.arange(0, 360, 30)

# Assign each wind direction to a sector
sprog['sector'] = pd.cut(sprog['wind_direction'], bins=bins, labels=labels[:-1], right=False)

# Handle the values from 345 to 360 (assign to sector 0)
sprog.loc[(sprog['wind_direction'] >= 345) | (sprog['wind_direction'] < 15), 'sector'] = 0

# Initialize list to store results
results = []

# Function to define equations for Weibull parameters
def equations(params, mean, variance):
    A, k = params
    eq1 = A * gamma(1 + 1/k) - mean
    eq2 = A**2 * (gamma(1 + 2/k) - (gamma(1 + 1/k))**2) - variance
    return (eq1, eq2)

# Calculate A and k for each sector
for sector in labels:
    sector_data = sprog[sprog['sector'] == sector]['wind_speed']
    
    if len(sector_data) > 0:
        # Calculate the first moment (mean) and second moment (variance)
        sprog_ws_mean = sector_data.mean()
        sprog_variance = sector_data.var(ddof=1)  # sample variance
        
        # Initial guesses for A and k
        initial_guess = [sprog_ws_mean, 1.5]
        
        # Solve the equations
        A_optimal, k_optimal = fsolve(equations, initial_guess, args=(sprog_ws_mean, sprog_variance))
        
        # Store results
        results.append({'sector': sector, 'A': A_optimal, 'k': k_optimal})

# Create a results DataFrame
results_df = pd.DataFrame(results)

# Plotting the Weibull PDF for each sector
x_values = np.linspace(0, sprog['wind_speed'].max(), 1000)

plt.figure(figsize=(15, 10))

for idx, row in results_df.iterrows():
    sector = row['sector']
    A = row['A']
    k = row['k']
    
    # Weibull PDF formula
    weibull_pdf = (k / A) * (x_values / A)**(k - 1) * np.exp(-(x_values / A)**k)
    
    plt.subplot(3, 4, idx + 1)
    plt.plot(x_values, weibull_pdf, label=f'Sector {sector} (A={A:.2f}, k={k:.2f})')
    
    # Plot actual wind speed data histogram for the sector
    sector_data = sprog[sprog['sector'] == sector]['wind_speed']
    plt.hist(sector_data, bins=15, density=True, alpha=0.5, color='grey', label='Data Histogram')
    
    plt.title(f'Sector {sector}')
    plt.xlabel('Wind Speed (m/s)')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.show()
